In [112]:
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
from scipy.stats import linregress
import matplotlib.pyplot as plt
import sys
import os

# =============================================================================
# MODELS
# =============================================================================

def model_1exp(t, A, tau, B, C):
    """Single exponential: i(t) = A·exp(-t/τ) + B/√t + C"""
    return A * np.exp(-t / tau) + B / np.sqrt(t) + C

# def model_2exp(t, A, tau, B, d, C):
#     """Single exponential: i(t) = A·exp(-t/τ) + B/t^d + C"""
#     return A * np.exp(-t / tau) + B / t**(d) + C

def model_2exp(t, A1, tau1, A2, tau2, C):
    """Double exponential: i(t) = A₁·exp(-t/τ₁) + A₂·exp(-t/τ₂) + C"""
    return A1 * np.exp(-t / tau1) + A2 * np.exp(-t / tau2) + C

def calculate_bic(n, k, ss_res):
    """Calculate BIC for model comparison."""
    if ss_res <= 0:
        return np.inf
    return n * np.log(ss_res / n) + k * np.log(n)
    
def round_to_discrete(arr_to_round, discrete_values):
    # Ensure discrete_values is a sorted NumPy array
    discrete_values = np.sort(np.unique(discrete_values))
    
    # Find the indices where elements should be inserted to maintain order (left side)
    # This effectively gives the index of the value just greater than or equal to the target
    indices = np.searchsorted(discrete_values, arr_to_round, side='left')
    
    # Clip indices to ensure they are within valid bounds (0 to len-1)
    # This handles values outside the range of discrete_values
    indices = np.clip(indices, 1, len(discrete_values) - 1)
    
    # Get the values at the left and right boundary indices
    left_values = discrete_values[indices - 1]
    right_values = discrete_values[indices]
    
    # Determine which value (left or right) is closer
    # A mask is created: True where the right value is closer or equal in distance
    # The mask determines if we subtract 1 from the index or keep it as is
    mask = np.abs(arr_to_round - left_values) > np.abs(arr_to_round - right_values)
    
    # Adjust indices based on the mask to select the nearest value's index
    nearest_indices = indices[mask]
    
    # Create the final rounded array by indexing the discrete values
    rounded_array = discrete_values[nearest_indices]
    # print(arr_to_round,discrete_values,rounded_array)
    return rounded_array
    
def detect_voltage_steps(df, voltage_steps, dfA):
    """Robustly detect voltage steps in noisy data."""
    voltage = df['Ewe/V'].values.copy()
    time = df['time/s'].values.copy()
    min_step_size=0.001
    min_duration=0.5

    # voltage_rounded = round_to_discrete(voltage, voltage_steps)
    voltage_rounded = np.round(voltage*100)/100
    # voltage_rounded_rhe = np.round(voltage*200)/200
    print(np.unique(voltage_rounded))
    # Identify distinct voltage levels
    levels = []
    current_level = voltage_rounded[0]
    level_start_idx = 0
    # print(np.unique(voltage_rounded))
    for i in range(1, len(voltage_rounded)):
        if abs(voltage_rounded[i] - current_level) > min_step_size / 2:
            duration = time[i-1] - time[level_start_idx]
            if duration >= min_duration:
                levels.append({
                    'V_mean': np.mean(voltage[level_start_idx:i]),
                    'V_meanR': np.mean(voltage_rounded[level_start_idx:i]),
                    'start_idx': level_start_idx,
                    'end_idx': i - 1,
                    'start_time': time[level_start_idx],
                    'end_time': time[i-1],
                    'duration': duration
                })
            current_level = voltage_rounded[i]
            level_start_idx = i
    
    # Last level
    duration = time[-1] - time[level_start_idx]
    if duration >= min_duration:
        levels.append({
            'V_mean': np.mean(voltage[level_start_idx:]),
            'V_meanR': np.mean(voltage_rounded[level_start_idx:]),
            'start_idx': level_start_idx,
            'end_idx': len(voltage) - 1,
            'start_time': time[level_start_idx],
            'end_time': time[-1],
            'duration': duration
        })

    print(f"\n  Detected {len(levels)} voltage levels:")
    for i, lvl in enumerate(levels):
        print(f"    Level {i+1}: {lvl['V_mean']:.3f}V for {lvl['duration']:.2f}s")
    
    # Identify transitions
    steps = []
    for i in range(1, len(levels)):
        prev_level = levels[i-1]
        curr_level = levels[i]
        dV = curr_level['V_mean'] - prev_level['V_mean']
        # dfA['iR-corrected E'][dfA['E(Anodic) applied']==1.361].iloc[0]
        # print(prev_level['V_meanR'],curr_level['V_meanR'])
        if prev_level['V_meanR']-voltage_rounded[1] > 0.009:
            vRb = dfA['iR-corrected E'][np.abs(dfA['E(Anodic) applied']-prev_level['V_meanR'])<0.009].iloc[0]
            charge = dfA['charge'][np.abs(dfA['E(Anodic) applied']-prev_level['V_meanR'])<0.009].iloc[0]
        else:
            vRb = prev_level['V_mean']
            charge = 0.0
        if curr_level['V_meanR']-voltage_rounded[1] > 0.009:
            vRa = dfA['iR-corrected E'][np.abs(dfA['E(Anodic) applied']-curr_level['V_meanR'])<0.009].iloc[0]
        else:
            vRa = curr_level['V_mean']

        # print(vRa, vRb)
        dV_RHE = vRa-vRb
        if abs(dV) >= min_step_size:
            steps.append({
                'step_num': len(steps) + 1,
                'V_before': prev_level['V_mean'],
                'V_before_iR': vRb,
                'V_after': curr_level['V_mean'],
                'V_after_iR': vRa,
                'dV': dV,
                'dV_iR': dV_RHE,
                'type': 'RISING' if dV > 0 else 'FALLING',
                't_start': curr_level['start_time'],
                't_end': curr_level['end_time'],
                'duration': curr_level['duration'],
                'charge': charge
            })


        
    print(f"\n  Found {len(steps)} voltage transitions")
    for i, step in enumerate(steps):
        print(f"    Step {i+1}: {step['V_before']:.3f}V to {step['V_after']:.3f}V - {step['V_before_iR']:.3f}V to {step['V_after_iR']:.3f}V")   
    return steps

# =============================================================================
# SINGLE TRANSIENT FITTING
# =============================================================================

def fit_transient(t, i, t_skip=0.002, t_fit_max=None):
    """Fit a single transient with 1-exp and 2-exp models."""
    t = np.asarray(t)
    i = np.asarray(i)
    
    # Filter data
    mask = t > t_skip
    if t_fit_max is not None:
        mask &= t <= t_fit_max
    
    t_fit = t[mask]
    i_fit = i[mask]
    
    if len(t_fit) < 20:
        return None
    
    n = len(t_fit)
    ss_tot = np.sum((i_fit - np.mean(i_fit))**2)
    
    # Initial guesses
    A0 = i_fit[0] - i_fit[-1]
    B0 = 0.1 * np.sign(A0) if A0 != 0 else 0.1
    C0 = i_fit[-1]
    
    # 1-EXPONENTIAL FIT
    try:
        popt_1, pcov_1 = curve_fit(
            model_1exp, t_fit, i_fit,
            p0=[A0, 0.1, B0, C0],
            bounds=([0, 0.00001, 0, -np.inf], [np.inf, 10, np.inf, np.inf]),
            maxfev=20000
        )
        perr_1 = np.sqrt(np.diag(pcov_1))
        
        i_pred_1 = model_1exp(t_fit, *popt_1)
        ss_res_1 = np.sum((i_fit - i_pred_1)**2)
        r2_1 = 1 - ss_res_1 / ss_tot
        bic_1 = calculate_bic(n, 4, ss_res_1)
        
        fit_1exp = {
            'A': popt_1[0], 'A_err': perr_1[0],
            'tau': popt_1[1], 'tau_err': perr_1[1],
            'B': popt_1[2], 'B_err': perr_1[2],
            'C': popt_1[3], 'C_err': perr_1[3],
            'R2': r2_1, 'BIC': bic_1,
            'i_pred': i_pred_1
        }
    except Exception as e:
        print(f"    1-exp fit failed: {e}")
        return None
    
    # 2-EXPONENTIAL FIT
    fit_2exp = None
    try:
        popt_2, pcov_2 = curve_fit(
            model_2exp, t_fit, i_fit,
            p0=[A0*0.6, 0.03, A0*0.4, 0.15, fit_1exp['C']],
            bounds=([0, 0.00001, 0, 0.00001,  -np.inf],
                   [np.inf, 10, np.inf, 2,  np.inf]),
            maxfev=30000
        )
        perr_2 = np.sqrt(np.diag(pcov_2))
        
        # Sort so tau1 < tau2
        A1, tau1, A2, tau2, C = popt_2
        A1_err, tau1_err, A2_err, tau2_err, C_err = perr_2
        
        if tau1 > tau2:
            A1, A2 = A2, A1
            tau1, tau2 = tau2, tau1
            A1_err, A2_err = A2_err, A1_err
            tau1_err, tau2_err = tau2_err, tau1_err
        
        i_pred_2 = model_2exp(t_fit, A1, tau1, A2, tau2, C)
        ss_res_2 = np.sum((i_fit - i_pred_2)**2)
        r2_2 = 1 - ss_res_2 / ss_tot
        bic_2 = calculate_bic(n, 6, ss_res_2)
        
        fit_2exp = {
            'A1': A1, 'A1_err': A1_err,
            'tau1': tau1, 'tau1_err': tau1_err,
            'A2': A2, 'A2_err': A2_err,
            'tau2': tau2, 'tau2_err': tau2_err,
            'C': C, 'C_err': C_err,
            'R2': r2_2, 'BIC': bic_2,
            'i_pred': i_pred_2
        }
    except Exception as e:
        print(f"    2-exp fit failed: {e}")
    
    # Model selection
    if fit_2exp is not None:
        delta_bic = fit_2exp['BIC'] - fit_1exp['BIC']
        best_model = '2exp' if delta_bic < -6 else '1exp'
    else:
        delta_bic = None
        best_model = '1exp'
    
    return {
        'fit_1exp': fit_1exp,
        'fit_2exp': fit_2exp,
        'best_model': best_model,
        'delta_BIC': delta_bic,
        't_fit': t_fit,
        'i_fit': i_fit
    }


dfA = pd.read_excel('iR_corrections.xlsx',sheet_name='OER')
print(dfA.head())
df = pd.read_csv('OER1_20241120_05_pulse_590mV_N_S1_02_CA_C02.txt', sep=r'\s+', decimal=',')
output_prefix = 'OER'

# dfA = pd.read_excel('iR_corrections.xlsx',sheet_name='CER 56mM')
# print(dfA.head())
# df = pd.read_csv('CER_56mM_20241120_32_pulse_590mV_RuO2_N_S1_02_CA_C01.txt', sep=r'\s+', decimal=',')
# output_prefix = 'CER_56mM'

# dfA = pd.read_excel('iR_corrections.xlsx',sheet_name='CER 166mM')
# print(dfA.head())
# df = pd.read_csv('CER_166mM_20241120_35_pulse_590mV_N_S1_CER_02_CA_C02.txt', sep=r'\s+', decimal=',')
# output_prefix = 'CER_166mM'

# dfA = pd.read_excel('iR_corrections.xlsx',sheet_name='CER 500mM')
# print(dfA.head())
# df = pd.read_csv('CER_500mM_20241120_22_pulse_590mV_RuO2_N_S1_02_CA_C01.txt', sep=r'\s+', decimal=',')
# output_prefix = 'CER_500mM'

# dfA = pd.read_excel('iR_corrections.xlsx',sheet_name='CER 1500mM')
# print(dfA['iR-corrected E'][dfA['E(Anodic) applied']==1.361].iloc[0])
# df = pd.read_csv('CER_1500mM_20241120_28_pulse_590mV_N_S1_CER_02_CA_C02.txt', sep=r'\s+', decimal=',')
# output_prefix = 'CER_1500mM'



csv_path = f'{output_prefix}_comprehensive_summary_a.csv'
print(df.head())
Vlow = 1.311 #V
Vref = 0.721 #V

t_col = 'time/s'
i_col = 'I/mA'
v_col = 'Ewe/V'

CBG = dfA['Current BG (at E(cathodic))'].iloc[0]
df[i_col] = -df[i_col].values - CBG
df[v_col] = df[v_col].values + Vref

voltage_steps = dfA['E(Anodic) applied'].values.copy() 
voltage_steps = np.append([1.301,1.311],voltage_steps)
print(voltage_steps,df.head())

steps = detect_voltage_steps(df, voltage_steps, dfA)
# print(steps)
steps = [s for s in steps if s['type'] == 'FALLING']
print(steps)

step_duration=4.0
t_skip=0.0015
t_fit_max=None

results = []
for step in steps:
    print(f"\nStep {step['step_num']}: {step['V_before']:.3f}V → {step['V_after']:.3f}V ({step['type']})")
    
    # Extract segment
    actual_duration = min(step['duration'], step_duration)
    t_end_fit = step['t_start'] + actual_duration
    
    mask = (df[t_col] >= step['t_start']) & (df[t_col] <= t_end_fit)
    df_seg = df[mask].copy()
    
    if len(df_seg) < 50:
        print("  Skipping: not enough points")
        continue
    
    t0 = df_seg[t_col].iloc[0]
    t = df_seg[t_col].values - t0
    current = df_seg[i_col].values
    
    # Fit
    fit_result = fit_transient(t, current, t_skip=t_skip, t_fit_max=t_fit_max)
    if fit_result is None:
        continue
    
    # Store result
    result = {
        'step': step['step_num'],
        'type': step['type'],
        'V_initial': step['V_before'],
        'V_initial_iR': step['V_before_iR'],
        'V_final': step['V_after'],
        'V_final_iR': step['V_after_iR'],
        'dV': abs(step['dV']),
        'dV_iR': abs(step['dV_iR']),
        't_start': step['t_start'],
        'duration': step['duration'],
        'charge': step['charge'],
        'fit_result': fit_result,
        't_data': t,
        'i_data': current
    }
    results.append(result)
    
    print(f"  1-exp: R²={fit_result['fit_1exp']['R2']:.5f}, A={fit_result['fit_1exp']['A']*1000:.1f}, B={fit_result['fit_1exp']['B']*1000:.1f}, τ={fit_result['fit_1exp']['tau']*1000:.1f}ms")
    if fit_result['fit_2exp']:
        print(f"  2-exp: R²={fit_result['fit_2exp']['R2']:.5f}, τ₁={fit_result['fit_2exp']['tau1']*1000:.1f}ms, τ₂={fit_result['fit_2exp']['tau2']*1000:.1f}ms")
        print(f"  Best: {fit_result['best_model'].upper()}")
    

rows = []

for res in results:
    f1 = res['fit_result']['fit_1exp']
    f2 = res['fit_result']['fit_2exp']
    dV = res['dV']
    dV_iR = res['dV_iR']
    # Calculate derived quantities for 1-exp
    Q_exp_1 = abs(f1['A']) * f1['tau'] * 1000  # mC (charge from exp term)
    B_over_dV_1 = abs(f1['B']) / dV if dV > 0 else np.nan
    
    row = {
        # Step info
        'Step': res['step'],
        'Type': res['type'],
        'V_initial': res['V_initial'],
        'V_initial_iR': res['V_initial_iR'],
        'V_final': res['V_final'],
        'V_final_iR': res['V_final_iR'],
        'dV': dV,
        'dV_iR': dV_iR,
        'duration': res['duration'],
        'charge': res['charge'],
        
        # 1-EXP MODEL PARAMETERS
        'A_1exp (mA)': f1['A'],
        'A_1exp_err': f1['A_err'],
        'tau_1exp (ms)': f1['tau'] * 1000,
        'tau_1exp_err (ms)': f1['tau_err'] * 1000,
        'B_1exp (mA·s^0.5)': f1['B'],
        'B_1exp_err': f1['B_err'],
        'C_1exp (mA)': f1['C'],
        'C_1exp_err': f1['C_err'],
        'R2_1exp': f1['R2'],
        'BIC_1exp': f1['BIC'],
        
        # 1-EXP DERIVED QUANTITIES
        '|A|_1exp': abs(f1['A']),
        '|B|_1exp': abs(f1['B']),
        '|B|/dV_1exp': B_over_dV_1,
        'Q_exp_1exp (mC)': Q_exp_1,
    }
    
    # 2-EXP MODEL PARAMETERS
    if f2 is not None:
        Q_fast = abs(f2['A1']) * f2['tau1'] * 1000  # mC
        Q_slow = abs(f2['A2']) * f2['tau2'] * 1000  # mC
        Q_total_2exp = Q_fast + Q_slow
        tau_ratio = f2['tau2'] / f2['tau1']
        A_ratio = abs(f2['A1']) / abs(f2['A2']) if abs(f2['A2']) > 0 else np.nan
        
        row.update({
            # Fast component (τ₁)
            'A1_2exp (mA)': f2['A1'],
            'A1_2exp_err': f2['A1_err'],
            'tau1_2exp (ms)': f2['tau1'] * 1000,
            'tau1_2exp_err (ms)': f2['tau1_err'] * 1000,
            
            # Slow component (τ₂)
            'A2_2exp (mA)': f2['A2'],
            'A2_2exp_err': f2['A2_err'],
            'tau2_2exp (ms)': f2['tau2'] * 1000,
            'tau2_2exp_err (ms)': f2['tau2_err'] * 1000,
            
            'C_2exp (mA)': f2['C'],
            'C_2exp_err': f2['C_err'],
            
            # Quality metrics
            'R2_2exp': f2['R2'],
            'BIC_2exp': f2['BIC'],
            'delta_BIC': res['fit_result']['delta_BIC'],
            'Best_Model': res['fit_result']['best_model'],
            
            # 2-EXP DERIVED QUANTITIES
            '|A1|_2exp': abs(f2['A1']),
            '|A2|_2exp': abs(f2['A2']),
            'tau2/tau1': tau_ratio,
            '|A1|/|A2|': A_ratio,
            'Q_fast (mC)': Q_fast,
            'Q_slow (mC)': Q_slow,
            'Q_total_2exp (mC)': Q_total_2exp,
            'Q_fast/Q_total': Q_fast / Q_total_2exp if Q_total_2exp > 0 else np.nan,
        })
    else:
        # Fill with NaN if 2-exp not available
        row.update({
            'A1_2exp (mA)': np.nan, 'A1_2exp_err': np.nan,
            'tau1_2exp (ms)': np.nan, 'tau1_2exp_err (ms)': np.nan,
            'A2_2exp (mA)': np.nan, 'A2_2exp_err': np.nan,
            'tau2_2exp (ms)': np.nan, 'tau2_2exp_err (ms)': np.nan,
            'C_2exp (mA)': np.nan, 'C_2exp_err': np.nan,
            'R2_2exp': np.nan, 'BIC_2exp': np.nan,
            'delta_BIC': np.nan, 'Best_Model': '1exp',
            '|A1|_2exp': np.nan, '|A2|_2exp': np.nan,
            'tau2/tau1': np.nan, '|A1|/|A2|': np.nan,
            'Q_fast (mC)': np.nan, 'Q_slow (mC)': np.nan,
            'Q_total_2exp (mC)': np.nan, 'Q_fast/Q_total': np.nan,
        })
    
    rows.append(row)

summary_df = pd.DataFrame(rows)

# Save to CSV

summary_df.to_csv(csv_path, index=False)
print(f"\nSaved: {csv_path}")

   E(Anodic) applied  Current Current BG (at E(cathodic))  iR correction  \
0              1.321  0.01900                      0.0188       0.000323   
1              1.341  0.01998                         NaN       0.000340   
2              1.361  0.01920                  Resistance       0.000326   
3              1.381  0.01970                         Ohm       0.000335   
4              1.401  0.02085                          17       0.000354   

   iR-corrected E   charge  
0        1.320677  0.01820  
1        1.340660  0.05327  
2        1.360674  0.09045  
3        1.380665  0.13040  
4        1.400646  0.17390  
      time/s     Ewe/V      I/mA
0  50.000399  0.589699  0.010509
1  50.000599  0.589279  0.034060
2  50.000799  0.589680  0.006054
3  50.000999  0.589394  0.013055
4  50.001199  0.589852  0.011782
[1.301 1.311 1.321 1.341 1.361 1.381 1.401 1.421 1.441 1.461 1.481 1.501
 1.521 1.541 1.561 1.581 1.601 1.631 1.661 1.691 1.721 1.751 1.781 1.811
 1.841 1.871 1.901 1.931 